In [ ]:
!pip install geemap
!pip install folium
!pip install earthengine-api --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 30.0 MB/s 
     |████████████████████████████████| 99 kB 10.5 MB/s 
     |████████████████████████████████| 1.2 MB 72.5 MB/s 
     |████████████████████████████████| 1.3 MB 71.7 MB/s 
     |████████████████████████████████| 130 kB 60.0 MB/s 
     |████████████████████████████████| 198 kB 83.9 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 46 kB 4.5 MB/s 
     |████████████████████████████████| 3.7 MB 49.8 MB/s 
     |████████████████████████████████| 98 kB 10.1 MB/s 
     |████████████████████████████████| 3.5 MB 63.7 MB/s 
     |████████████████████████████████| 128 kB 104.9 MB/s 
     |████████████████████████████████| 1.6 MB 62.1 MB/s 
     |████████████████████████████████| 79 kB 11.3 MB/s 
  Created wheel for ee-extra: filename=ee_extra-0.0.14-py3-none-any.whl size=209116 sha256=bb4f

In [ ]:
import folium
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=oFigQniis8hFZBZJ3VVWEd6LCUIhkZ00xxHljlbyuhQ&tc=ajIPefdjhinsbaw6QZzrIBdD_cbjBEpARANp8_4dQcw&cc=fRJ-t7YuE5n1_LAKJRIRMb9EP9Oc0zoB36gaipoj60s

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvtXdBv0MMVRv0m6aI_DUlPFVn0lbidSEvom1CbKa9stwN2_RXuudEQ

Successfully saved authorization token.


In [ ]:
# Map = geemap.Map()
# Map

In [ ]:
# Function to mask clouds based on the pixel_qa band of Landsat 4, 5 and 7 SR data.

def cloudMaskL457(image):
  qa = image.select('pixel_qa')
  cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
  # Get the pixel QA band.
  mask = image.mask().reduce(ee.Reducer.min())
  return image.updateMask(cloud.Not()).updateMask(mask)

### <font style='color:rgb(50,120,230)'>Setting the parameters for getting data from Google Earth Engine.</font>

In [ ]:
# The Region of interest, Cancún, Mexoco.
geometry = ee.Geometry.Polygon(
    [[  [-87.10988867671762,21.07449634501289],
        [-86.81669104488168,21.07449634501289],
        [-86.81669104488168,21.251229199817406],
        [-87.10988867671762,21.251229199817406]]], 
        None,
        False)

# Setting the parameters for Landsat5
visParams5 = {
  "bands": ['B3', 'B2', 'B1'],
  "min": 0,
  "max": 3000,
  "gamma": 1.4
}

dataset85 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1985-01-01', '1985-12-31').map(cloudMaskL457)
lansat85 = dataset85.median().clip(geometry)
dataset93 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1993-01-01', '1993-12-31').map(cloudMaskL457)
lansat93 = dataset93.median().clip(geometry)
dataset01 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2001-01-01', '2001-12-31').map(cloudMaskL457)
lansat01 = dataset01.median().clip(geometry)
dataset11 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2009-01-01', '2011-12-31').map(cloudMaskL457)
lansat11 = dataset11.median().clip(geometry)

# Using folium for getting an interactive map 

<hr style='border:none; height: 4px; background-color:#D3D3D3'/>

## Displaying Earth Engine image tiles to folium map
**`add_ee_layer()`** adds a method for displaying Earth Engine image tiles to folium map.

### Documentation
[`add_ee_layer()`](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)

## Drawing map
**`folium.map`** is the class for drawing maps.

### Documentation
[`folium.map`](http://python-visualization.github.io/folium/modules.html#module-folium.map)

<hr style='border:none; height: 4px; background-color:#D3D3D3'/>



In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

### <font style='color:rgb(50,120,230)'>Generating the Map with set center and zoom levels.</font>

In [ ]:

map = folium.Map(location = (21.18, -86.95), zoom_start = 11)
# Adding the generated images onto the map.
map.add_ee_layer(lansat85, visParams5, '1985')
map.add_ee_layer(lansat93, visParams5, '1993')
map.add_ee_layer(lansat01, visParams5, '2001')
map.add_ee_layer(lansat11, visParams5, '2011')

# Add a layer control panel to the map.
map.add_child(folium.LayerControl())

# Display the map.
display(map)

In [ ]:
task_config = {
  'folder': '/content/drive/MyDrive/LandSatImages/',
  'scale': 30,
  'region': geometry,
  'fileFormat': 'GeoTIFF',
  'maxPixels': 1e9,
  'formatOptions': {
    'cloudOptimized': True
  }
}
task1 = ee.batch.Export.image.toDrive(lansat85, description= '1985api', **task_config)
task2 = ee.batch.Export.image.toDrive(lansat93, description= '1993api', **task_config)
task3 = ee.batch.Export.image.toDrive(lansat01, description= '2001api', **task_config)
task4 = ee.batch.Export.image.toDrive(lansat11, description= '2011api', **task_config)

In [ ]:
task1.start()
task2.start()
task3.start()
task4.start()

In [ ]:
!pip install nasapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
